# Ollama LLM - Thinking model

이 예제는 Ollama를 이용해서 local 환경에서 다양한 오픈 소스 모델을 서빙할 수 있는 예제입니다.
사용자들은 ollama 사이트에서 특정 오픈소스 모델을 검색해서 사용할 수 있습니다.

* Ollama Website
  * https://ollama.com/search


## Ollama 다운로드 및 설치

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


### Ollama API 서버 실행
다른 프로그램(애플리케이션)이 모델을 사용할 수 있도록 백그라운드에서 API 서버를 실행

아래 터미널을 클릭해서 창을 열고 아래 명령어를 입력해주세요.
```
ollama serve
```

위의 명령어를 실행했을때 로그가 에러없이 올라가면 정상적으로 ollama API 서버가 실행 된것입니다.

### Ollama 실행 및 모델 다운로드
#### Ollama command 명령어

In [ ]:
!ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model
  show        Show information for a model
  run         Run a model
  stop        Stop a running model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


#### 다운로드 받은 모델 리스트 확인

In [ ]:
!ollama list

NAME         ID              SIZE      MODIFIED       
gemma3:1B    8648f39daa8f    815 MB    21 minutes ago    


#### 모델 다운로드

In [ ]:
!ollama pull gemma3:4B

In [ ]:
!ollama list

NAME         ID              SIZE      MODIFIED       
gemma3:4B    a2af6cc3eb7f    3.3 GB    26 seconds ago    
gemma3:1B    8648f39daa8f    815 MB    28 minutes ago    


## Thinking

Ollama의 모델은 "사고"를 지원합니다. 사고는 최종 답변을 제공하기 전에 응답에 대해 추론하고 숙고하는 과정입니다.

아래에서는 thinking 매개변수와 qwen3:8b 모델을 사용하여 스트리밍 모드와 비스트리밍 모드 모두에서 Ollama 모델에서 사고를 활성화하는 방법을 보여줍니다.

In [ ]:
!ollama pull qwen3:8b

In [ ]:
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="qwen3:8b",
    request_timeout=360,
    thinking=True,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [ ]:
resp = llm.complete("What is 434 / 22?")

In [ ]:
print(resp.additional_kwargs["thinking"])

In [ ]:
print(resp.text)

In [ ]:
resp_gen = llm.stream_complete("What is 434 / 22?")

thinking_started = False
response_started = False

for resp in resp_gen:
    if resp.additional_kwargs.get("thinking_delta", None):
        if not thinking_started:
            print("\n\n-------- Thinking: --------\n")
            thinking_started = True
            response_started = False
        print(resp.additional_kwargs["thinking_delta"], end="", flush=True)
    if resp.delta:
        if not response_started:
            print("\n\n-------- Response: --------\n")
            response_started = True
            thinking_started = False
        print(resp.delta, end="", flush=True)